In [ ]:
!pip install python-dotenv # como instalo dotenv?
!pip install langchain
!pip install langchain-openai
!pip install langchain-chroma
!pip install wikipedia
!pip install pypdf
!pip install pinecone
!pip install faiss-cpu
!pip install lark
!pip install rank_bm25

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
# Se cargan las credenciales

dotenv_path = "/content/drive/My Drive/ANALITICA_NOVIEMBRE/LANGCHAIN/.env"
load_dotenv(dotenv_path)
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

In [ ]:
llm = ChatOpenAI()
output = llm.invoke("Cual es la capital de Argentina") # model = "gpt-3.5-turbo"
print(output.content)

La capital de Argentina es Buenos Aires.


**La intención es generar una concatenación de pasos a seguir**

# 1- SIMPLE CHAINS

## A- LLMChain para texto

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain # Chain to run queries against LLMs.

In [ ]:
llm = ChatOpenAI()

In [ ]:
template = """
You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.
"""

prompt_template = PromptTemplate.from_template(template = template)

prompt = prompt_template.format(virus = "ebola", language = "romanian")

print(prompt)


You are an experienced virologist.
Write a few sentences about the following ebola in romanian.



In [ ]:
chain  = LLMChain(
    llm = llm,
    prompt = prompt_template
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
output = chain.invoke({"virus": "HIV", "language": "Spanish"}) # Si solo requiere un input, entonces se puede poner un str directamente, en vez de un dict.

In [ ]:
output # El output es un dict que contiene los inputs y el output con una clave llamada "text"

{'virus': 'HIV',
 'language': 'Spanish',
 'text': 'El VIH, o virus de inmunodeficiencia humana, es un virus que ataca al sistema inmunológico, debilitando la capacidad del cuerpo para combatir infecciones y enfermedades. Se transmite principalmente a través de relaciones sexuales sin protección, el uso de agujas contaminadas, o de madre a hijo durante el embarazo, parto o lactancia. Aunque actualmente no existe una cura para el VIH, se puede controlar con medicamentos antirretrovirales que permiten a las personas infectadas llevar una vida saludable y prolongada.'}

In [ ]:
output["text"]

'El VIH, o virus de inmunodeficiencia humana, es un virus que ataca al sistema inmunológico, debilitando la capacidad del cuerpo para combatir infecciones y enfermedades. Se transmite principalmente a través de relaciones sexuales sin protección, el uso de agujas contaminadas, o de madre a hijo durante el embarazo, parto o lactancia. Aunque actualmente no existe una cura para el VIH, se puede controlar con medicamentos antirretrovirales que permiten a las personas infectadas llevar una vida saludable y prolongada.'

* A diferencia de usar `llm.invoke()`, al usar una cadena como en este caso `LLMChain` nos permite acceder a atributos adicionales como `verbose` y `memory`.
* Esto es útil para `debbugear`.
* Utilizando `verbose = True`

In [ ]:
chain  = LLMChain(
    llm = llm,
    prompt = prompt_template,
    verbose = True
)

output = chain.invoke({"virus": "HIV", "language": "Spanish"})

print("Imprimiendo la respuesta:")
print(output["text"])



> Entering new LLMChain chain...
Prompt after formatting:

You are an experienced virologist.
Write a few sentences about the following HIV in Spanish.


> Finished chain.
Imprimiendo la respuesta:
El VIH (Virus de Inmunodeficiencia Humana) es un virus que ataca el sistema inmunológico del ser humano, específicamente a los linfocitos T CD4. Esto debilita la capacidad del cuerpo para combatir infecciones y enfermedades, lo que eventualmente puede llevar al desarrollo del SIDA (Síndrome de Inmunodeficiencia Adquirida). Aunque no existe una cura para el VIH, los avances en la medicina han permitido el desarrollo de tratamientos efectivos que pueden controlar la replicación del virus y mejorar la calidad de vida de las personas infectadas.


**`Completion` vs `Chat`**

In [ ]:
chat = ChatOpenAI()

template = """
How do you say good morning in {language}?
"""

prompt_template = PromptTemplate.from_template(
    template = template
)

prompt = prompt_template.format(language = "spanish")

print("Prompt:")
print(prompt)

chain_chat  = LLMChain(
    llm = chat,
    prompt = prompt_template
    )

output = chain_chat.invoke({"language": "Spanish"})

print("Imprimiendo la respuesta:")
print(output["text"])

Prompt:

How do you say good morning in spanish?

Imprimiendo la respuesta:
"Buenos días"


In [ ]:
completion = OpenAI()

template = """
How do you say good morning in {language}?
"""

prompt_template = PromptTemplate.from_template(
    template = template
)

prompt = prompt_template.format(language = "spanish")

print("Prompt:")
print(prompt)

chain_chat  = LLMChain(
    llm = completion,
    prompt = prompt_template
    )

output = chain_chat.invoke({"language": "Spanish"})

print("Imprimiendo la respuesta:")
print(output["text"])

Prompt:

How do you say good morning in spanish?

Imprimiendo la respuesta:

Buenos días.


## B- LLMChain para otros casos de uso

In [ ]:
from sklearn.datasets import fetch_openml

# Descargar el dataset de Titanic
titanic = fetch_openml(name = "titanic", version = 1, as_frame = True)

import pandas as pd
titanic_df = pd.DataFrame(titanic["data"], columns = titanic["feature_names"])
titanic_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,None,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"


In [ ]:
template = """
Tienes la siguiente información {context}, que contiene datos sobre los pasajeros del Titanic, incluyendo características como clase de pasajero, nombre, sexo, edad, número de hermanos/cónyuges a bordo, número de padres/hijos a bordo, tarifa, cabina, puerto de embarque, bote de rescate (si aplicable), y si se recuperó el cuerpo.
Debes responder a la siguiente pregunta {question}.
Limita tu respuesta a la entrega de la respuesta.
Respuesta en español.
"""

prompt_template = PromptTemplate(
    template = template,
    input_variables = ["context", "question"],
)

chain = LLMChain(llm = completion, prompt = prompt_template, verbose = True)

output = chain.invoke({"context": titanic_df, "question": "¿Cuál es el promedio exacto de edad de las mujeres y redondeado a 2 decimales?"})



> Entering new LLMChain chain...
Prompt after formatting:

Tienes la siguiente información       pclass                                             name     sex  \
0        1.0                    Allen, Miss. Elisabeth Walton  female   
1        1.0                   Allison, Master. Hudson Trevor    male   
2        1.0                     Allison, Miss. Helen Loraine  female   
3        1.0             Allison, Mr. Hudson Joshua Creighton    male   
4        1.0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   
...      ...                                              ...     ...   
1304     3.0                             Zabour, Miss. Hileni  female   
1305     3.0                            Zabour, Miss. Thamine  female   
1306     3.0                        Zakarian, Mr. Mapriededer    male   
1307     3.0                              Zakarian, Mr. Ortin    male   
1308     3.0                               Zimmerman, Mr. Leo    male   

          age  sibsp  parch  t

In [ ]:
output["text"]

'\nLa edad promedio exacta de las mujeres en el Titanic es de 28.69 años.'

In [ ]:
round(titanic_df[titanic_df["sex"] == "female"]["age"].mean(), 2)

28.69

# 2- SEQUENTIAL CHAINS

**Con `SequentialChain` el output de una primer cadena funciona como el input de una segunda cadena.**

## A- Primer ejemplo

In [ ]:
from langchain.chains import LLMChain, SimpleSequentialChain

In [ ]:
llm1 = ChatOpenAI(temperature = 0.5)

template1 = """
You are an experienced scientist and Python programmer. Write a function in Python that implements the concept of a {concept}.
"""

prompt_template1 = PromptTemplate.from_template(template = template1)

chain1 = LLMChain(llm = llm1, prompt = prompt_template1)

In [ ]:
llm2 = ChatOpenAI(temperature = 1)

template2 = """
Given the Python function {function}, describe it as detailed as possible.
"""

prompt_template2 = PromptTemplate.from_template(template = template2)

chain2 = LLMChain(llm = llm2, prompt = prompt_template2)

In [ ]:
overall_chain = SimpleSequentialChain(chains = [chain1, chain2], verbose = True)
output = overall_chain.invoke("Linear Regression")



> Entering new SimpleSequentialChain chain...
Sure! Here is an example of a simple linear regression function in Python:

```python
import numpy as np

def linear_regression(X, y):
    # Calculate the mean of X and y
    mean_x = np.mean(X)
    mean_y = np.mean(y)

    # Calculate the terms needed for the numerator and denominator of the slope
    numerator = np.sum((X - mean_x) * (y - mean_y))
    denominator = np.sum((X - mean_x) ** 2)

    # Calculate the slope (m) and intercept (b) of the regression line
    m = numerator / denominator
    b = mean_y - m * mean_x

    return m, b

# Example usage
X = np.array([1, 2, 3, 4, 5])
y = np.array([2, 3, 4, 5, 6])

slope, intercept = linear_regression(X, y)
print("Slope:", slope)
print("Intercept:", intercept)
```

This function takes in two numpy arrays `X` and `y`, representing the independent and dependent variables, respectively. It calculates the slope and intercept of the regression line using the least squares method and returns th

In [ ]:
output

{'input': 'Linear Regression',
 'output': 'The `linear_regression` function begins by importing the `numpy` library as `np`. It defines a function `linear_regression` that takes in two numpy arrays `X` and `y`.\n\nThe function first calculates the mean of `X` and `y` using `np.mean()`. It then calculates the numerator and denominator needed for calculating the slope of the regression line.\n\nThe numerator is computed by taking the sum of the product of the differences between each element of `X` and the mean of `X`, and the differences between each element of `y` and the mean of `y`. The denominator is computed by taking the sum of the squared differences between each element of `X` and the mean of `X`.\n\nNext, the function calculates the slope `m` by dividing the numerator by the denominator, and the intercept `b` of the regression line using the formula `b = mean_y - m * mean_x`.\n\nFinally, the function returns the slope `m` and intercept `b`.\n\nAn example usage of the function i

## B- Segundo ejemplo

**Como vemos, la primer salida tiene como salida predeterminada a `text`:**

In [ ]:
template = """
Write a very short {language} function that will {task}.
"""

prompt_template = PromptTemplate.from_template(
    template = template
)

chain_chat  = LLMChain(
    llm = chat,
    prompt = prompt_template
    )

output = chain_chat.invoke({"language": "python", "task": "Return a list of numbers from 1 to 10."})

In [ ]:
output

{'language': 'python',
 'task': 'Return a list of numbers from 1 to 10.',
 'text': 'def get_numbers():\n    return list(range(1, 11))'}

In [ ]:
output["text"]

'def get_numbers():\n    return list(range(1, 11))'

**Podemos cambiar el nombre de la variable de salida, con el argumento `output_key`:**

In [ ]:
template = """
Write a very short {language} function that will {task}.
"""

prompt_template = PromptTemplate.from_template(
    template = template
)

chain_chat  = LLMChain(
    llm = chat,
    prompt = prompt_template,
    output_key = "code"
    )

output = chain_chat.invoke({"language": "python", "task": "Return a list of numbers from 1 to 10."})

In [ ]:
output

{'language': 'python',
 'task': 'Return a list of numbers from 1 to 10.',
 'code': 'def generate_numbers():\n    return list(range(1, 11))'}

In [ ]:
test_template = """
Write a test for the following {language} code:\n{code}.?
"""

test_prompt_template = PromptTemplate.from_template(
    template = test_template
)

test_chain_chat  = LLMChain(
    llm = chat,
    prompt = test_prompt_template,
    output_key = "test"
    )

In [ ]:
output = test_chain_chat.invoke({"language": "python", "code": "def get_numbers():\n    return list(range(1, 11))."})

In [ ]:
output

{'language': 'python',
 'code': 'def get_numbers():\n    return list(range(1, 11)).',
 'test': "import unittest\n\ndef get_numbers():\n    return list(range(1, 11))\n\nclass TestGetNumbers(unittest.TestCase):\n    \n    def test_output_type(self):\n        result = get_numbers()\n        self.assertIsInstance(result, list)\n        \n    def test_output_length(self):\n        result = get_numbers()\n        self.assertEqual(len(result), 10)\n        \n    def test_output_values(self):\n        result = get_numbers()\n        expected_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\n        self.assertEqual(result, expected_values)\n        \nif __name__ == '__main__':\n    unittest.main()"}

In [ ]:
from langchain.chains import LLMChain, SequentialChain

In [ ]:
chain = SequentialChain(
    chains = [chain_chat, test_chain_chat],
    input_variables = ["language", "task"],
    output_variables = ["code", "test"]
)

In [ ]:
output = chain.invoke({"language": "python", "task": "Return a list of numbers from 1 to 10."})

In [ ]:
output

{'language': 'python',
 'task': 'Return a list of numbers from 1 to 10.',
 'code': 'def generate_numbers():\n    return list(range(1, 11))',
 'test': "import unittest\n\ndef generate_numbers():\n    return list(range(1, 11))\n\nclass TestGenerateNumbers(unittest.TestCase):\n\n    def test_generate_numbers_length(self):\n        result = generate_numbers()\n        self.assertEqual(len(result), 10)\n\n    def test_generate_numbers_values(self):\n        result = generate_numbers()\n        expected_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\n        self.assertEqual(result, expected_values)\n\nif __name__ == '__main__':\n    unittest.main()"}

In [ ]:
print(">>>>>> GENERATED CODE:")
print(output["code"])
print()
print(">>>>>> GENERATED TEST:")
print(output["test"])

>>>>>> GENERATED CODE:
def generate_numbers():
    return list(range(1, 11))

>>>>>> GENERATED TEST:
import unittest

def generate_numbers():
    return list(range(1, 11))

class TestGenerateNumbers(unittest.TestCase):

    def test_generate_numbers_length(self):
        result = generate_numbers()
        self.assertEqual(len(result), 10)

    def test_generate_numbers_values(self):
        result = generate_numbers()
        expected_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        self.assertEqual(result, expected_values)

if __name__ == '__main__':
    unittest.main()


# 3- Y LA MEMORIA?

**Lamentablemente, los LLM no tiene memoria por sí solos...**

In [ ]:
template = """
Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Aquí está la pregunta del usuario:
###
{pregunta}
###
"""

prompt_template = PromptTemplate.from_template(template = template)

prompt = prompt_template.format(pregunta = "¿Quién es Messi?")

print(prompt)


Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Aquí está la pregunta del usuario:
###
¿Quién es Messi?
###



In [ ]:
chain  = LLMChain(
    llm = llm,
    prompt = prompt_template,
    verbose = True
)

In [ ]:
output = chain.invoke({"pregunta": "¿Quién es Messi?"})



> Entering new LLMChain chain...
Prompt after formatting:

Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Aquí está la pregunta del usuario:
###
¿Quién es Messi?
###


> Finished chain.


In [ ]:
output["text"]

'Lionel Messi es un famoso futbolista argentino considerado como uno de los mejores jugadores de fútbol de todos los tiempos. Ha jugado la mayor parte de su carrera en el FC Barcelona, donde ha ganado numerosos títulos y premios individuales. Messi también ha sido capitán de la selección argentina y ha tenido una carrera exitosa a nivel internacional.'

In [ ]:
output = chain.invoke({"pregunta": "¿Cuántas copas del mundo ganó?"})



> Entering new LLMChain chain...
Prompt after formatting:

Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Aquí está la pregunta del usuario:
###
¿Cuántas copas del mundo ganó?
###


> Finished chain.


In [ ]:
output["text"]

'Hasta la fecha, la selección nacional de fútbol de Brasil ha ganado un total de 5 Copas del Mundo. ¡Es un récord en la historia del fútbol! ¿Hay algo más en lo que pueda ayudarte?'

In [ ]:
output = chain.invoke({"pregunta": "¿Pero de quién estamos hablando?"})



> Entering new LLMChain chain...
Prompt after formatting:

Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Aquí está la pregunta del usuario:
###
¿Pero de quién estamos hablando?
###


> Finished chain.


In [ ]:
output["text"]

'Estamos hablando de ti, el usuario. ¿En qué puedo ayudarte hoy?'

**Veamos un pequeño ejemplo, iremos más en profundidad en unos minutos y en otra notebook.**

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(memory_key = "chat_history")

In [ ]:
template = """
Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Estas son las conversaciones anteriores:
###
{chat_history}
###
Aquí está la pregunta del usuario:
###
{pregunta}
###
"""

prompt_template = PromptTemplate.from_template(template = template)

In [ ]:
chain  = LLMChain(
    llm = llm,
    prompt = prompt_template,
    verbose = True,
    memory = memory
)

In [ ]:
output = chain.invoke({"pregunta": "¿Quién es Messi?"})



> Entering new LLMChain chain...
Prompt after formatting:

Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Estas son las conversaciones anteriores:
###

###
Aquí está la pregunta del usuario:
###
¿Quién es Messi?
###


> Finished chain.


In [ ]:
output["text"]

'Lionel Messi es un futbolista argentino considerado uno de los mejores jugadores de fútbol de todos los tiempos. Ha jugado la mayor parte de su carrera en el FC Barcelona y actualmente juega para el Paris Saint-Germain. Messi ha ganado numerosos premios y récords, y es conocido por su habilidad, visión de juego y capacidad goleadora. ¿Hay algo más que te gustaría saber sobre él?'

In [ ]:
output = chain.invoke({"pregunta": "¿Cuántas copas del mundo ganó?"})



> Entering new LLMChain chain...
Prompt after formatting:

Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Estas son las conversaciones anteriores:
###
Human: ¿Quién es Messi?
AI: Lionel Messi es un futbolista argentino considerado uno de los mejores jugadores de fútbol de todos los tiempos. Ha jugado la mayor parte de su carrera en el FC Barcelona y actualmente juega para el Paris Saint-Germain. Messi ha ganado numerosos premios y récords, y es conocido por su habilidad, visión de juego y capacidad goleadora. ¿Hay algo más que te gustaría saber sobre él?
###
Aquí está la pregunta del usuario:
###
¿Cuántas copas del mundo ganó?
###


> Finished chain.


In [ ]:
output["text"]

'Lionel Messi no ha ganado ninguna copa del mundo con la selección argentina. Sin embargo, ha tenido una exitosa carrera a nivel de clubes y ha ganado múltiples títulos con el FC Barcelona y actualmente con el Paris Saint-Germain. ¿Hay algo más en lo que pueda ayudarte?'

In [ ]:
output = chain.invoke({"pregunta": "¿Pero de quién estamos hablando?"})



> Entering new LLMChain chain...
Prompt after formatting:

Eres una Inteligencia Artificial muy amigable!
Responde a las preguntas del usuario amablemente.
Estas son las conversaciones anteriores:
###
Human: ¿Quién es Messi?
AI: Lionel Messi es un futbolista argentino considerado uno de los mejores jugadores de fútbol de todos los tiempos. Ha jugado la mayor parte de su carrera en el FC Barcelona y actualmente juega para el Paris Saint-Germain. Messi ha ganado numerosos premios y récords, y es conocido por su habilidad, visión de juego y capacidad goleadora. ¿Hay algo más que te gustaría saber sobre él?
Human: ¿Cuántas copas del mundo ganó?
AI: Lionel Messi no ha ganado ninguna copa del mundo con la selección argentina. Sin embargo, ha tenido una exitosa carrera a nivel de clubes y ha ganado múltiples títulos con el FC Barcelona y actualmente con el Paris Saint-Germain. ¿Hay algo más en lo que pueda ayudarte?
###
Aquí está la pregunta del usuario:
###
¿Pero de quién estamos hablando?

In [ ]:
output["text"]

'Estamos hablando de Lionel Messi, el futbolista argentino considerado uno de los mejores jugadores de fútbol de todos los tiempos. ¿Hay algo más que te gustaría saber sobre él?'

In [ ]:
memory.load_memory_variables({})

{'chat_history': 'Human: ¿Quién es Messi?\nAI: Lionel Messi es un futbolista argentino considerado uno de los mejores jugadores de fútbol de todos los tiempos. Ha jugado la mayor parte de su carrera en el FC Barcelona y actualmente juega para el Paris Saint-Germain. Messi ha ganado numerosos premios y récords, y es conocido por su habilidad, visión de juego y capacidad goleadora. ¿Hay algo más que te gustaría saber sobre él?\nHuman: ¿Cuántas copas del mundo ganó?\nAI: Lionel Messi no ha ganado ninguna copa del mundo con la selección argentina. Sin embargo, ha tenido una exitosa carrera a nivel de clubes y ha ganado múltiples títulos con el FC Barcelona y actualmente con el Paris Saint-Germain. ¿Hay algo más en lo que pueda ayudarte?\nHuman: ¿Pero de quién estamos hablando?\nAI: Estamos hablando de Lionel Messi, el futbolista argentino considerado uno de los mejores jugadores de fútbol de todos los tiempos. ¿Hay algo más que te gustaría saber sobre él?'}